In [ ]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_las_out_full_path, delimiter=';')

In [2]:
wells_list = las_data['well_name'].value_counts().index.tolist()
len(wells_list)

223

In [3]:
las_data_cutted = pd.DataFrame()

for well in wells_list:    
    well_las_data = las_data[las_data['well_name'] == well]
    cuted_well_las, not_empty = main.cut_data_frame_by_satur(well_las_data)
    if not_empty:
         las_data_cutted = las_data_cutted.append(cuted_well_las)

las_data_cutted.describe()

,STRT,STOP,DEPT,aps,rp,kp,kgl,kpr,kvo,kng,lit,satur
count,308924.000000,308924.000000,308924.000000,127606.000000,188709.000000,191843.000000,182965.000000,191968.000000,177069.000000,308924.000000,308924.000000,308924.000000
mean,2495.859367,3045.657912,2779.719434,0.391987,15.011739,0.134285,0.241503,28.421996,0.802113,0.040608,0.309309,1.511475
std,787.343964,421.575971,603.898441,0.322511,141.618242,0.065662,0.116804,123.826467,0.272653,0.137920,0.462210,2.823551
min,860.000000,1776.600000,860.000000,0.000000,0.300000,0.002000,0.041000,0.000000,0.204000,0.000000,0.000000,0.000000
25%,2473.400000,2844.400000,2661.900000,0.100000,4.300000,0.092000,0.144000,0.100000,0.494000,0.000000,0.000000,0.000000
50%,2756.400000,3055.400000,2894.800000,0.320000,6.100000,0.125000,0.233000,0.100000,1.000000,0.000000,0.000000,0.000000
75%,2999.400000,3296.200000,3163.700000,0.690000,8.900000,0.161000,0.340000,3.600000,1.000000,0.000000,1.000000,2.000000
max,3466.000000,3936.000000,3901.500000,1.000000,8635.100000,0.368000,0.465000,1759.700000,1.000000,0.796000,1.000000,10.000000


In [4]:
# загружаем wc
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
water_content_data.head()

,well_name,date_start,wc
0,380,04.07.1905,0.49
1,379,01.12.2006,0.30
2,370,01.02.2007,0.13
3,417,01.03.2007,0.08
4,389,01.08.2007,0.22


In [5]:
las_data_cutted = pd.merge(las_data_cutted, water_content_data, on=main.keys_dict[main.kid_well])
las_data_cutted.describe()

,STRT,STOP,DEPT,aps,rp,kp,kgl,kpr,kvo,kng,lit,satur,wc
count,186019.000000,186019.000000,186019.000000,92161.000000,115987.000000,116519.000000,112164.000000,116696.000000,112100.000000,186019.000000,186019.000000,186019.000000,186019.000000
mean,2818.209337,3063.790583,2952.850520,0.391633,16.285179,0.111362,0.234785,3.589508,0.813063,0.052313,0.232401,1.088776,0.306624
std,294.285366,251.133307,281.590501,0.321144,69.939827,0.040077,0.120887,17.434522,0.260000,0.157094,0.422364,2.481312,0.171276
min,2086.200000,2717.200000,2086.200000,0.000000,0.300000,0.002000,0.041000,0.000000,0.204000,0.000000,0.000000,0.000000,0.010000
25%,2610.800000,2836.000000,2749.000000,0.100000,5.200000,0.082000,0.130000,0.100000,0.508000,0.000000,0.000000,0.000000,0.180000
50%,2810.400000,3035.000000,2927.600000,0.320000,6.900000,0.113000,0.231000,0.100000,1.000000,0.000000,0.000000,0.000000,0.280000
75%,2999.400000,3239.600000,3147.000000,0.690000,10.000000,0.139000,0.335000,2.500000,1.000000,0.000000,0.000000,0.000000,0.410000
max,3466.000000,3644.200000,3643.100000,1.000000,3740.200000,0.257000,0.450000,381.200000,1.000000,0.796000,1.000000,10.000000,0.900000


In [ ]:
def f_cube(x, a, b, c, d):
    x2 = x * x
    return a * x * x2 + b * x2 + c * x + d 

In [ ]:
well_name = '531'
data_well =  las_data_cutted[las_data_cutted['well_name'] == well_name] 

In [ ]:
import scipy.optimize as optim
x_arr = data_well['DEPT'].tolist()
y_arr = data_well['kng'].tolist()

In [ ]:
# нелинейный мнк
popt, pcov = optim.curve_fit(f_cube, x_arr, y_arr)
for x, y in zip(x_arr, y_arr):
    f_calc = f_cube(x, *popt)


In [ ]:
# кубический сплайн
from scipy import interpolate
f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'cubic')
for x, y in zip(x_arr, y_arr):
    f_calc = f_spline(x)

In [36]:
well_name_list = las_data_cutted['well_name'].value_counts().index.tolist()
data_well =  las_data_cutted[las_data_cutted['well_name'] == '554']
x_arr = data_well['DEPT'].tolist()
y_arr = data_well['kng'].tolist()
from scipy import interpolate
f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear') # 'cubic'

In [79]:
from scipy import interpolate
step_count = 100;
well_name_list = las_data_cutted['well_name'].value_counts().index.tolist()
x_values = []
y_values = []
for well_name in well_name_list:
    data_well =  las_data_cutted[las_data_cutted['well_name'] == well_name]
    x_arr = data_well['DEPT'].tolist()
    y_arr = data_well['kng'].tolist()
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well['DEPT'].min()
    h_end = data_well['DEPT'].max()
    h_step = (h_end - h_start)/step_count
    x_temp = []
    i = 0
    while(i < step_count):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['wc'].tolist()[0])
print('end fill')

end fill


In [83]:
y_values_r = [int(10 * y_val) for y_val in y_values]

In [91]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(x_values, y_values_r, test_size=0.3, random_state=17)

In [94]:
tree = DecisionTreeClassifier(max_depth=20, random_state=99)
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_hold)
accuracy_score(y_hold, tree_pred)

0.16

In [96]:
from sklearn.model_selection import GridSearchCV, cross_val_score
print(step_count)
tree_params = {'max_depth': range(1,30), 'max_features': range(1, step_count)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(x_train, y_train)

100
Fitting 5 folds for each of 2871 candidates, totalling 14355 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done 699 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 9699 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 14355 out of 14355 | elapsed:   15.7s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=99, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 30), 'max_features': range(1, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [97]:
tree_grid.best_params_

{'max_depth': 7, 'max_features': 23}

In [98]:
tree_grid.best_score_

0.31034482758620691

In [99]:
accuracy_score(y_hold, tree_grid.predict(x_hold))

0.20000000000000001

Настройка случайного леса

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=99)
print(np.mean(cross_val_score(forest, x_train, y_train, cv=5))) # 0.949

forest_params = {'max_depth': range(1,20),
'max_features': range(4,len(x_train[0]))}

forest_grid = GridSearchCV(forest, forest_params,
cv=5, n_jobs=-1, verbose=True)
 
forest_grid.fit(x_train, y_train)
# forest_grid.best_params_, forest_grid.best_score_ # ({'max_depth': 9, 'max_features': 6}, 0.951)
# accuracy_score(y_holdout, forest_grid.predict(X_holdout)) # 0.953